## Data Collection Code from MAST

In [ ]:
import sys,os,glob
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import extract_array
from astropy.coordinates import SkyCoord
from astropy import wcs
from astropy.wcs.utils import skycoord_to_pixel
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astropy.visualization import (simple_norm,LinearStretch)
import pandas as pd
import subprocess

In [ ]:
#My very bad way of getting things done
subprocess.call("cd ..", shell = True)

In [ ]:
#Open and read the unmatched pantheon dataset
df = pd.read_csv('data_files/unmatched_pantheon.csv')
vp = pd.read_csv('data_files/valid_points_index_list.csv')
print(vp.dtypes)

In [ ]:
resolved_coord_index = 12
print(df['resolved_coord'][resolved_coord_index]) 

obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][resolved_coord_index],
                                            radius="0.006 deg",
                                            intentType = 'science',
                                            filters = ['F1*'],
                                            obs_collection=['HST'])
obs_table = obs_table[obs_table['calib_level']==3]
data_products = Observations.get_product_list(obs_table)
data_products = data_products[data_products['calib_level'] == 3]
data_products = data_products[data_products['productType'] == 'SCIENCE']
print(data_products)

In [ ]:
#Creating a function that will create a folder for the data (named the resolved coordinate) and keep the query there

def save_images_supernova(resolved_coord_index):
    """
    If there are images for the supernovae in JWST or HST at the resolved query, create a folder and keep the data there
    Parameters:
        resolved_coord_index: integer, the index number of the query
    Returns:
        Folder of Data at requested file path (pantheon_data_folder/{resolved coord})
    """
    print(resolved_coord_index, df['resolved_coord'][resolved_coord_index])

    #try the the query, if a error is thrown then print "No Data Points" and "skip" this data point, else make folder and store data
    try:
        obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][resolved_coord_index],
                                            radius="0.006 deg",
                                            intentType = 'science',
                                            filters = ['F1*'],
                                            obs_collection=['HST'])
        obs_table = obs_table[obs_table['calib_level']==3]
        data_products = Observations.get_product_list(obs_table)
        data_products = data_products[data_products['calib_level'] == 3]
        data_products = data_products[data_products['productType'] == 'SCIENCE']
    except: 
        print("No Data Points :(")
        exit
    else:
        print("Has Data Points!")
        folder_path = "pantheon_data_folder/{}".format(df['SNID'][resolved_coord_index])
        os.makedirs(folder_path, exist_ok=True)
        manifest = Observations.download_products(data_products, download_dir=folder_path, extension=['fits'])
        print(manifest)

In [ ]:
for i in range(len(vp['0'])):
    save_images_supernova(vp['0'][i])